In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.layers import Dense,Flatten
from keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import keras

In [62]:
base_model= InceptionV3(input_shape=(256,256,3),include_top=False)

In [63]:
for layer in base_model.layers:
    layer.trainable=False

In [64]:
X=Flatten()(base_model.output)
X=Dense(units=2, activation='sigmoid')(X)

model=Model(base_model.input, X)
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 127, 127, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 127, 127, 32) 96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 127, 127, 32) 0           batch_normalization_282[0][0]    
____________________________________________________________________________________________

In [65]:
model.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9, decay = 0.005), 
             loss = "sparse_categorical_crossentropy",
             metrics = ["accuracy"])

In [66]:
datasets_directory = "../input/dogs-and-cats-image-classification/dataset/"

In [67]:
batch_size = 36

In [68]:
train_dataGenerator = ImageDataGenerator(rescale = 1./255,
                                         
                                         shear_range = 0.4, 
                                         zoom_range=0.4,
                                         width_shift_range=0.4,
                                         height_shift_range=0.4, 
                                         fill_mode="nearest"
                                        )

validation_dataGenerator = ImageDataGenerator(rescale = 1./255)

In [69]:
# Load the datasets from the folder
trainDatasets = train_dataGenerator.flow_from_directory(datasets_directory+"/training_set", 
                                                       target_size=(256, 256),
                                                       batch_size=batch_size, 
                                                       class_mode='binary')

validationDatasets = validation_dataGenerator.flow_from_directory(datasets_directory+"/test_set", 
                                                       target_size=(256, 256),
                                                       batch_size=batch_size, 
                                                       class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [70]:
classInformation = trainDatasets.class_indices
print("Class Information: {}".format(classInformation))
classes = list(classInformation)
print("Class Names: {}".format(classes))

Class Information: {'cats': 0, 'dogs': 1}
Class Names: ['cats', 'dogs']


In [71]:
classSamplesT = trainDatasets.samples
print("Training Data Samples: {}".format(classSamplesT))
classSamplesV = validationDatasets.samples
print("Validation Data Samples: {}".format(classSamplesV))

Training Data Samples: 8000
Validation Data Samples: 2000


In [72]:
from tensorflow.keras.callbacks import ModelCheckpoint
weghtPath = "best_weights_9.hdf5"
checkPoint = ModelCheckpoint(weghtPath, monitor="val_acc", verbose = 1, save_best_only = True, save_weghts_only = True, mode = "max")
callBacksList = [checkPoint]

In [74]:
history = model.fit(trainDatasets,
                    steps_per_epoch = classSamplesT//batch_size, 
                    validation_data=validationDatasets,
                    epochs=20, 
                    validation_steps=classSamplesV//batch_size, 
                    callbacks=callBacksList)

Epoch 1/20
  3/222 [..............................] - ETA: 8:06 - loss: 0.3481 - accuracy: 0.9259

KeyboardInterrupt: 